In [1]:
import rebound
import pandas as pd
from rebound import InterruptiblePool
import numpy as np
import os
import time

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import time 
from scipy import stats
import rebound
from scipy.special import gamma
from scipy.stats import norm, ks_2samp
from celmech.disturbing_function import *
from sympy import symbols, init_printing,Function,simplify,collect,cos
init_printing()
import numpy as np
from celmech.poincare import Poincare, PoincareHamiltonian
from celmech import Andoyer, AndoyerHamiltonian
from celmech.resonances import resonant_period_ratios, resonance_intersections_list, resonance_pratio_span
from rebound.plotting import OrbitPlotOneSlice
import itertools
import time
path = '/mnt/raid-cita/dtamayo/stabilitydataset/data/random/simulation_archives/runs/'

In [3]:
def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

In [4]:
def worker(df):
    Ndump=1
    for i in range(df.shape[0]):
        try:
            run(df, i, tmax=1.e4)
        except:
            for col in columns:
                df.loc[df.index[i], col] = np.nan
        if i%Ndump == 0:
            df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [5]:
def findres(sim, i1, i2):
    delta = 0.03
    maxorder = 2
    ps = Poincare.from_Simulation(sim=sim).particles # get averaged mean motions
    n1 = ps[i1].n
    n2 = ps[i2].n
    
    m1 = ps[i1].m
    m2 = ps[i2].m
    
    Pratio = n2/n1
    res = resonant_period_ratios(Pratio-delta,Pratio+delta, order=maxorder)
    
    Z = np.sqrt((ps[i1].e*np.cos(ps[i1].pomega) - ps[i2].e*np.cos(ps[i2].pomega))**2 + (ps[i1].e*np.sin(ps[i1].pomega) - ps[i2].e*np.sin(ps[i2].pomega))**2)
    
    maxstrength = 0
    j, k, i1, i2, strength = -1, -1, -1, -1, -1
    for a, b in res:
        s = np.abs(np.sqrt(m1+m2)*Z**((b-a)/2.)/(b*n2 - a*n1))
        #print('{0}:{1}'.format(b, a), (b*n2 - a*n1), s)
        if s > maxstrength:
            j = b
            k = b-a
            i1 = 1
            i2 = 2
            strength=s
            maxstrength = s
            
    return j, k, strength

In [6]:
def run(df, index, tmax=1.e4, runcel=False):
    t0 = time.time()
    file = path+"sa{0}".format(df.loc[df.index[index], 'runstring'])
    sim = rebound.Simulation.from_file(file)
    ps = sim.particles
    sim.init_megno()
    
    N = sim.N - sim.N_var
    a0 = [0] + [sim.particles[i].a for i in range(1, N)]
    Npairs = int((N-1)*(N-2)/2)
    js, ks, strengths = np.zeros(Npairs, dtype=np.int), np.zeros(Npairs, dtype=np.int), np.zeros(Npairs)
    maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength = -1, -1, -1, -1, -1, -1

    Zcross = np.zeros(Npairs)
    #print('pairindex, i1, i2, j, k, strength')
    for i, [i1, i2] in enumerate(itertools.combinations(np.arange(1, N), 2)):
        js[i], ks[i], strengths[i] = findres(sim, i1, i2)
        Zcross[i] = (ps[int(i2)].a-ps[int(i1)].a)/ps[int(i1)].a
        #print(i, i1, i2, js[i], ks[i], strengths[i])
        if strengths[i] > maxstrength:
            maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength = js[i], ks[i], i1, i2, i, strengths[i]
    
    df.loc[df.index[index], 'Zcross12'] = Zcross[0]
    df.loc[df.index[index], 'Zcross13'] = Zcross[1]
    df.loc[df.index[index], 'Zcross23'] = Zcross[2]
    df.loc[df.index[index], 'maxj'] = maxj
    df.loc[df.index[index], 'maxk'] = maxk
    df.loc[df.index[index], 'maxi1'] = maxi1
    df.loc[df.index[index], 'maxi2'] = maxi2
    df.loc[df.index[index], 'maxstrength'] = maxstrength
    
    sortedstrengths = strengths.copy()
    sortedstrengths.sort()
    if sortedstrengths[-1] > 0 and sortedstrengths[-2] > 0:
        df.loc[df.index[index], 'secondres'] = sortedstrengths[-2]/sortedstrengths[-1]
    else:
        df.loc[df.index[index], 'secondres'] = -1
        
    if maxstrength > 0 and runcel is True:
        #print(maxj, maxk, maxi1, maxi2, maxstrength)
        andvars = Andoyer.from_Simulation(sim=sim, j=maxj, k=maxk, a10=a0[maxi1], i1=maxi1, i2=maxi2)
        H = AndoyerHamiltonian(andvars)
        #print(maxj, maxk, maxi1, maxi2, H.state.phi)
    else:
        H = None

    #print('max', maxi1, maxi2, maxj, maxk, maxpairindex, maxstrength)
    #print('df (j, k, pairindex):', df.loc[df.index[index], 'j'], df.loc[df.index[index], 'k'], df.loc[df.index[index], 'pairindex'])

    Nout = 1000
    times = np.linspace(0, tmax, Nout)

    eminus = np.zeros((Npairs, Nout))
    rebound_Z, rebound_phi = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    rebound_Zcom, rebound_phiZcom = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    rebound_Zstar, rebound_dKprime = np.zeros((Npairs,Nout)), np.zeros((Npairs,Nout))
    celmech_Z, celmech_phi = np.zeros(Nout), np.zeros(Nout)
    celmech_Zcom, celmech_phiZcom = np.zeros(Nout), np.zeros(Nout)
    celmech_Zstar, celmech_dKprime = np.zeros(Nout), np.zeros(Nout)

    for i,t in enumerate(times):
        for j, [i1, i2] in enumerate(itertools.combinations(np.arange(1, N), 2)):
            i1, i2 = int(i1), int(i2)
            eminus[j, i] = np.sqrt((ps[i2].e*np.cos(ps[i2].pomega)-ps[i1].e*np.cos(ps[i1].pomega))**2 + (ps[i2].e*np.sin(ps[i2].pomega)-ps[i1].e*np.sin(ps[i1].pomega))**2)
            if js[j] != -1:
                avars = Andoyer.from_Simulation(sim=sim, j=js[j], k=ks[j], a10=a0[i1], i1=i1, i2=i2)
                rebound_Z[j, i] = avars.Z
                rebound_phi[j, i] = avars.phi
                rebound_Zcom[j, i] = avars.Zcom
                rebound_phiZcom[j, i] = avars.phiZcom
                rebound_Zstar[j, i] = avars.Zstar
                rebound_dKprime[j, i] = avars.dKprime
        
        if maxstrength > 0 and runcel is True:
            celmech_Z[i] = H.state.Z
            celmech_phi[i] = H.state.phi
            celmech_Zcom[i] = H.state.Zcom
            celmech_phiZcom[i] = H.state.phiZcom
            celmech_Zstar[i] = H.state.Zstar
            celmech_dKprime[i] = H.state.dKprime

            H.integrate(t)
        try:
            sim.integrate(t)
        except:
            break
        
    mask = eminus[0] > 0 # where there are data points in case sim ends early
    times = times[mask]
    eminus = eminus[:, mask]
    rebound_Z, rebound_phi = rebound_Z[:, mask], rebound_phi[:, mask]
    rebound_Zcom, rebound_phiZcom =  rebound_Zcom[:, mask], rebound_phiZcom[:, mask]
    rebound_Zstar, rebound_dKprime = rebound_Zstar[:, mask], rebound_dKprime[:, mask]
    celmech_Z, celmech_phi, celmech_Zcom, celmech_phiZcom = celmech_Z[mask], celmech_phi[mask], celmech_Zcom[mask], celmech_phiZcom[mask]
    celmech_Zstar, celmech_dKprime = celmech_Zstar[mask], celmech_dKprime[mask]
    
    for i, s in zip([0,2], ['12', '23']): # take adjacent ones
        EM = eminus[i]
        Zc = Zcross[i]
        df.loc[df.index[index], 'EMmed'+s] = np.median(EM)/Zc
        df.loc[df.index[index], 'EMmax'+s] = EM.max()/Zc
        p = np.poly1d(np.polyfit(times, EM, 3))
        m = p(times)
        EMdrift = np.abs((m[-1]-m[0])/m[0])
        df.loc[df.index[index], 'EMdrift'+s] = EMdrift
        maxindex = (m == m.max()).nonzero()[0][0] # index where cubic polynomial fit to EM reaches max to track long wavelength variations (secular?)
        if EMdrift > 0.1 and (maxindex < 0.01*Nout or maxindex > 0.99*Nout): # don't flag as not capturing secular if Z isn't varying significantly in first place
            df.loc[df.index[index], 'capseculartscale'+s] = 0
        else:
            df.loc[df.index[index], 'capseculartscale'+s] = 1
        df.loc[df.index[index], 'EMdetrendedstd'+s] = pd.Series(EM-m).std()/EM[0]
        rollstd = pd.rolling_std(EM, window=100)
        df.loc[df.index[index], 'EMrollingstd'+s] = np.median(rollstd[100:])/EM[0]
        var = [EM[:j].var() for j in range(len(EM))]
        p = np.poly1d(np.polyfit(times[len(var)//2:], var[len(var)//2:], 1)) # fit only second half to get rid of transient
        df.loc[df.index[index], 'DiffcoeffEM'+s] = p[1]/Zc**2
        df.loc[df.index[index], 'medvarEM'+s] = np.median(var[len(var)//2:])/Zc**2
        if strengths[i] != -1:
            Z = rebound_Z[i]
            df.loc[df.index[index], 'Zmed'+s] = np.median(Z)/Zc
            df.loc[df.index[index], 'Zmax'+s] = rebound_Z[i].max()/Zc
            p = np.poly1d(np.polyfit(times, Z, 3))
            m = p(times)
            df.loc[df.index[index], 'Zdetrendedstd'+s] = pd.Series(Z-m).std()/Z[0]
            rollstd = pd.rolling_std(Z, window=100)
            df.loc[df.index[index], 'Zrollingstd'+s] = np.median(rollstd[100:])/Z[0]
            var = [Z[:j].var() for j in range(len(Z))]
            p = np.poly1d(np.polyfit(times[len(var)//2:], var[len(var)//2:], 1)) # fit only second half to get rid of transient
            df.loc[df.index[index], 'DiffcoeffZ'+s] = p[1]/Zc**2
            df.loc[df.index[index], 'medvarZ'+s] = np.median(var[len(var)//2:])/Zc**2
            df.loc[df.index[index], 'Zcomdrift'+s] = np.max(np.abs(rebound_Zcom[i]-rebound_Zcom[i, 0])/rebound_Zcom[i, 0])
            rollstd = pd.rolling_std(rebound_Zcom[i], window=100)
            df.loc[df.index[index], 'Zcomrollingstd'+s] = np.median(rollstd[100:])/rebound_Zcom[i,0]
            df.loc[df.index[index], 'phiZcomdrift'+s] = np.max(np.abs(rebound_phiZcom[i]-rebound_phiZcom[i, 0]))
            rollstd = pd.rolling_std(rebound_phiZcom[i], window=100)
            df.loc[df.index[index], 'phiZcomrollingstd'+s] = np.median(rollstd[100:])
            df.loc[df.index[index], 'Zstardrift'+s] = np.max(np.abs(rebound_Zstar[i]-rebound_Zstar[i, 0])/rebound_Zstar[i, 0])
            rollstd = pd.rolling_std(rebound_Zcom[i], window=100)
            df.loc[df.index[index], 'Zstarrollingstd'+s] = np.median(rollstd[100:])/rebound_Zstar[i,0]
            Zcosphi = Z*np.cos(rebound_phi[i])
            df.loc[df.index[index], 'Zcosphistd'+s] = Zcosphi.std()/Zc
            df.loc[df.index[index], 'medZcosphi'+s] = np.median(Zcosphi)/Zc
    df.loc[df.index[index], 'tlyap'] = 1/sim.calculate_lyapunov()
    df.loc[df.index[index], 'featgentime'] = time.time()-t0
    return sim, times, maxj, maxk, maxi1, maxi2, maxpairindex, maxstrength, Npairs, js, ks, strengths, eminus, rebound_Z, rebound_phi, rebound_Zcom, rebound_phiZcom, rebound_Zstar, rebound_dKprime, celmech_Z, celmech_phi, celmech_Zcom, celmech_phiZcom, celmech_Zstar, celmech_dKprime, Zcross

In [7]:
columns = ['featgentime', 'Zcross12', 'Zcross13', 'Zcross23', 'maxj', 'maxk', 'maxi1', 'maxi2', 'maxstrength', 'secondres', 'tlyap']
for i, s in zip([0,2], ['12', '23']): # take adjacent ones
    columns += ['EMmed'+s, 'EMmax'+s, 'EMdrift'+s, 'capseculartscale'+s, 'EMdetrendedstd'+s, 'EMrollingstd'+s, 'DiffcoeffEM'+s, 'medvarEM'+s, 'Zmed'+s, 'Zmax'+s, 'Zdetrendedstd'+s, 'Zrollingstd'+s, 'DiffcoeffZ'+s, 'medvarZ'+s, 'Zcomdrift'+s, 'Zcomrollingstd'+s, 'phiZcomdrift'+s, 'phiZcomrollingstd'+s, 'Zstardrift'+s, 'Zstarrollingstd'+s, 'Zcosphistd'+s, 'medZcosphi'+s]
columns

['featgentime',
 'Zcross12',
 'Zcross13',
 'Zcross23',
 'maxj',
 'maxk',
 'maxi1',
 'maxi2',
 'maxstrength',
 'secondres',
 'tlyap',
 'EMmed12',
 'EMmax12',
 'EMdrift12',
 'capseculartscale12',
 'EMdetrendedstd12',
 'EMrollingstd12',
 'DiffcoeffEM12',
 'medvarEM12',
 'Zmed12',
 'Zmax12',
 'Zdetrendedstd12',
 'Zrollingstd12',
 'DiffcoeffZ12',
 'medvarZ12',
 'Zcomdrift12',
 'Zcomrollingstd12',
 'phiZcomdrift12',
 'phiZcomrollingstd12',
 'Zstardrift12',
 'Zstarrollingstd12',
 'Zcosphistd12',
 'medZcosphi12',
 'EMmed23',
 'EMmax23',
 'EMdrift23',
 'capseculartscale23',
 'EMdetrendedstd23',
 'EMrollingstd23',
 'DiffcoeffEM23',
 'medvarEM23',
 'Zmed23',
 'Zmax23',
 'Zdetrendedstd23',
 'Zrollingstd23',
 'DiffcoeffZ23',
 'medvarZ23',
 'Zcomdrift23',
 'Zcomrollingstd23',
 'phiZcomdrift23',
 'phiZcomrollingstd23',
 'Zstardrift23',
 'Zstarrollingstd23',
 'Zcosphistd23',
 'medZcosphi23']

In [8]:
featuresfile='randomshortfeatures.csv'

In [15]:
df = pd.read_csv('/mnt/raid-cita/dtamayo/stabilitydataset/data/random/random.csv', index_col=0)
df['SimID'] = np.arange(df.shape[0])
#df['Stable'] = df['instability_time'] >= 9.99e8
for col in columns:
    df[col] = 0
df.set_index('SimID', inplace=True)
df.tail()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
24995,0024995.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24996,0024996.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24997,0024997.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24998,0024998.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24999,0024999.bin,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df.to_csv(featuresfile, encoding='ascii')

In [9]:
Nworkers=60
df = pd.read_csv(featuresfile, index_col=0)
df.tail()

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
24995,0024995.bin,1.153346,0.171907,0.387978,0.184376,9.0,2.0,2.0,3.0,0.003431,...,1.745551e-06,0.000717,0.060754,0.005449,0.016285,0.002086,0.084769,0.001440,0.367484,0.084805
24996,0024996.bin,2.964576,0.118284,0.220488,0.091394,8.0,1.0,2.0,3.0,0.006414,...,4.867103e-07,0.002863,1.386549,0.311781,6.123500,2.000274,0.752978,0.014807,0.123352,-0.057720
24997,0024997.bin,2.349224,0.579004,0.697671,0.075153,2.0,1.0,1.0,2.0,0.019195,...,-1.135500e-07,0.001531,0.053027,0.011746,0.574987,0.021306,1.199827,0.018510,0.096530,-0.112411
24998,0024998.bin,2.939841,0.055424,0.286817,0.219241,4.0,1.0,2.0,3.0,0.003782,...,1.357478e-08,0.000079,0.125268,0.004959,0.336359,0.010575,9.060472,0.015823,0.091789,0.004391
24999,0024999.bin,2.884900,0.316695,0.849302,0.404503,3.0,1.0,1.0,2.0,0.002190,...,1.085541e-11,0.000023,0.210186,0.039263,0.573982,0.045580,0.208377,0.003629,0.008251,0.000276


In [10]:
df = df[df['featgentime'] == 0.]
print(df.shape)
IDs = df.index.values
splits = chunkify(IDs, Nworkers)
dfs = [df.loc[split] for split in splits]

(39, 56)


In [ ]:
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have cha

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might 

/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/raid-cita/dtamayo/rebound/rebound/simulation.py:369: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary 

/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:31: RuntimeWarning: invalid value encountered in sqrt
  Xstarres = -0.5*np.sqrt(3.*Phiprime+2.)
/mnt/raid-cita/dtamayo/celmech/celmech/poincare.py:116: RuntimeWarning: invalid value encountered in sqrt
  sLambda = np.sqrt(sim.G*M*orb.a)
/mnt/raid-cita/dtamayo/celmech/celmech/poincare.py:117: RuntimeWarning: invalid value encountered in sqrt
  sGamma = sLambda*(1.-np.sqrt(1.-orb.e**2))
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:270: RuntimeWarning: invalid value encountered in double_scalars
  andvars.dKprime = ((j-k)*p['m2']*p['sLambda20']*dL2hat + j*p['m1']*p['sLambda10']*dL1hat)/p['K0']
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/

/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:270: RuntimeWarning: invalid value encountered in double_scalars
  andvars.dKprime = ((j-k)*p['m2']*p['sLambda20']*dL2hat + j*p['m1']*p['sLambda10']*dL1hat)/p['K0']
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Phiprime)**(-1.5)))
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:270: RuntimeWarning: invalid value encountered in double_scalars
  andvars.dKprime = ((j-k)*p['m2']*p['sLambda20']*dL2hat + j*p['m1']*p['sLambda10']*dL1hat)/p['K0']
/mnt/raid-cita/dtamayo/celmech/celmech/andoyer.py:29: RuntimeWarning: invalid value encountered in double_scalars
  Xstarres = np.sqrt(-Phiprime)*np.sinh(1./3.*np.arcsinh(-abs(Ph

/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:125: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:127: RuntimeWarning: Degrees of freedom <= 0 for slice
/mnt/raid-cita/dtamayo/j5/lib/python3.4/site-packages/ipykernel_launcher.py:132: FutureWarning: pd.rolling_std is deprecated for ndarrays and will be removed in a future version
/

In [71]:
df = pd.read_csv('/mnt/raid-cita/dtamayo/newstabilitydataset/data/csvs/data.csv', index_col=0)
df['SimID'] = np.arange(df.shape[0])
df['Stable'] = df['instability_time'] >= 9.99e8
df = df[['SimID', 'Stable', 'instability_time', 'runstring', 'seed']]
for col in columns:
    df[col] = 0
df.set_index('SimID', inplace=True)
df = df.iloc[:10]
df.head()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import os

df = pd.read_csv(featuresfile, index_col=0)

for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        df.update(newdf)
        
df[df['featgentime'] == 0.].shape

124802.csv
124818.csv
124807.csv
124770.csv
124812.csv
124803.csv
124810.csv
124816.csv
124815.csv
124817.csv
124813.csv
124814.csv
124799.csv
124771.csv
124806.csv
124819.csv
124804.csv
124780.csv
124775.csv
124821.csv
124798.csv
124782.csv
124809.csv
124765.csv
124805.csv
124773.csv
124822.csv
124811.csv
124766.csv
124808.csv


In [10]:
df.to_csv(featuresfile, encoding='ascii')

In [ ]:
df[df['SimID']]

In [14]:
df[df['featgentime'] == 0]

,runstring,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,maxi1,maxi2,maxstrength,...,DiffcoeffZ23,medvarZ23,Zcomdrift23,Zcomrollingstd23,phiZcomdrift23,phiZcomrollingstd23,Zstardrift23,Zstarrollingstd23,Zcosphistd23,medZcosphi23
SimID,,,,,,,,,,,,,,,,,,,,,
703,0000703.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
714,0000714.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1113,0001113.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1524,0001524.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1716,0001716.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1824,0001824.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4263,0004263.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4661,0004661.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4771,0004771.bin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [192]:
test = pd.read_csv('test.csv', index_col=0)
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,True,9.990000e+08,0009995.bin,1409995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,False,3.814426e+06,0009996.bin,2809996,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,False,1.498363e+04,0009997.bin,1409997,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,False,9.419807e+05,0009998.bin,709998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,False,1.895071e+07,0009999.bin,9999,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [191]:
import os

#test = pd.DataFrame(columns=test.columns)
for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        test.update(newdf)
        
#test = test.sort_values('SimID')
#test = test.reset_index(drop=True)
df[df['featgentime'] == 0.]



24103.csv
23373.csv
24110.csv
24098.csv
23372.csv
24350.csv
24339.csv
24343.csv
24337.csv
24108.csv
23375.csv
23138.csv
23371.csv
24340.csv
24109.csv
24112.csv
24349.csv
24097.csv
24107.csv
24111.csv
24101.csv
23141.csv
23148.csv
23136.csv
23384.csv
24338.csv
24344.csv
24336.csv
24345.csv
23377.csv
24351.csv
24106.csv
23150.csv
24341.csv
23381.csv
23376.csv
24346.csv
23380.csv
24342.csv
23379.csv
24348.csv
24105.csv
23135.csv
24099.csv
23144.csv
23385.csv
23134.csv
23370.csv
24104.csv
24102.csv
23146.csv
23383.csv
24347.csv
23140.csv
23142.csv


,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
0,False,1.545872e+06,0000000.bin,3500000.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
1,True,9.990000e+08,0000001.bin,700001.0,14.756695,0.240504,0.340424,0.080547,9.0,1.0,...,0.779525,0.797556,0.008385,0.005011,1.420017e-09,0.000092,0.0,0.0,0.0,0.023081
2,True,9.990000e+08,0000002.bin,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
3,False,2.287671e+06,0000003.bin,2100003.0,3.098584,0.406112,1.007603,0.427768,5.0,2.0,...,0.115293,0.150077,0.004127,0.001260,5.512634e-08,0.000386,0.0,0.0,0.0,0.039200
4,False,9.668931e+05,0000004.bin,700004.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
5,False,5.755894e+05,0000005.bin,1400005.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
6,False,7.248372e+07,0000006.bin,6.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
7,False,2.225858e+04,0000007.bin,2100007.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000
8,False,2.930966e+05,0000008.bin,8.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.0,0.0,0.000000


In [100]:
test[test['featgentime']==0].shape

In [99]:
test.to_csv('newshortfeatures.csv')

In [61]:
df.to_csv('shortfeatures.csv', encoding='ascii')

In [171]:
test.to_csv('test.csv', encoding='ascii')

In [186]:
test = pd.read_csv('test.csv', index_col=0)
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
9995,True,9.990000e+08,0009995.bin,1409995,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,False,3.814426e+06,0009996.bin,2809996,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,False,1.498363e+04,0009997.bin,1409997,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,False,9.419807e+05,0009998.bin,709998,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9999,False,1.895071e+07,0009999.bin,9999,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
test.shape

In [193]:
test = pd.read_csv('test.csv', index_col=0)
test = test.iloc[:100]
test.tail()

,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,maxk,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
SimID,,,,,,,,,,,,,,,,,,,,,
95,False,3.023697e+04,0000095.bin,95,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
96,False,1.661314e+07,0000096.bin,96,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,False,1.017439e+07,0000097.bin,97,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
98,True,1.000000e+09,0000098.bin,1400098,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,False,4.521761e+04,0000099.bin,99,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
test.head()

,SimID,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
0,0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
test['SimID'].values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [44]:
def worker(df):
    Ndump=10
    for i, row in df.iterrows():
        run(df, row['SimID'])
        if i%Ndump == 0:
            df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')
    df.to_csv('tempcsvs/'+str(os.getpid())+'.csv', encoding='ascii')

In [48]:
def run(df, ID):
    if ID % 2 == 0:
        df.loc[df['SimID']==ID, 'featgentime'] = 7

In [300]:
test.head()

,SimID,Stable,instability_time,runstring,seed,featgentime,Zcross12,Zcross13,Zcross23,maxj,...,Zmed23,Zmax23,Zdetrendedstd23,Zrollingstd23,DiffcoeffZ23,medvarZ23,deltaZcom23,deltaphiZcom23,deltaZstar23,Zcosphistd23
0,0,False,1.545872e+06,0000000.bin,3500000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,True,9.990000e+08,0000001.bin,700001,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,True,9.990000e+08,0000002.bin,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,False,2.287671e+06,0000003.bin,2100003,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,False,9.668931e+05,0000004.bin,700004,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
Nworkers=3
test = test[test['featgentime'] == 0]
print(test.shape)
IDs = test['SimID'].values
splits = chunkify(IDs, Nworkers)
dfs = [test[test['SimID'].isin(split)] for split in splits]
pool = InterruptiblePool(Nworkers)
pool.map(worker, dfs)

(33, 35)


[None, None, None]

In [84]:
import os

test = pd.DataFrame(columns=test.columns)
for f in os.listdir('tempcsvs/'):
    if f[-4:] == '.csv':
        print(f)
        newdf = pd.read_csv('tempcsvs/'+f, index_col=0)
        test = test.append(newdf)
        
test = test.sort_values('SimID')
test = test.reset_index(drop=True)
test.head()

10377.csv
10381.csv
10371.csv
10375.csv
10385.csv
10376.csv
10384.csv
10386.csv
10379.csv
10383.csv
10382.csv
10378.csv
10373.csv


,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0


In [88]:
test.head(50)

,DiffcoeffZ12,DiffcoeffZ23,SimID,Stable,Zcosphistd12,Zcosphistd23,Zcross12,Zcross13,Zcross23,Zdetrendedstd12,...,maxi2,maxj,maxk,maxstrength,medvarZ12,medvarZ23,runstring,secondres,secondstrength,seed
0,0.000000e+00,-5.249295e-10,0.0,False,0.000000,0.012035,0.504063,0.594658,0.060234,0.000000,...,3.0,12.0,1.0,0.048244,0.000000,1.471706e-04,0000000.bin,0.195509,0.0,3500000.0
1,5.530677e-11,9.176673e-11,2.0,True,0.003070,0.000785,0.129660,1.261558,1.001981,0.003071,...,2.0,6.0,1.0,0.067762,0.000009,3.508728e-07,0000002.bin,0.029946,0.0,2.0
2,-1.838390e-08,-5.059799e-09,4.0,False,0.023602,0.033524,0.059897,0.332922,0.257596,0.022934,...,2.0,12.0,1.0,0.030171,0.000586,1.147590e-03,0000004.bin,0.140455,0.0,700004.0
3,2.151018e-08,-1.581556e-07,5.0,False,0.011011,0.053589,0.185250,0.259467,0.062617,0.001381,...,3.0,23.0,2.0,0.001197,0.000029,2.971640e-03,0000005.bin,0.567226,0.0,1400005.0
4,0.000000e+00,-5.992155e-10,6.0,False,0.000000,0.009534,0.516020,0.760215,0.161076,0.000000,...,3.0,5.0,1.0,0.111359,0.000000,8.956325e-05,0000006.bin,-1.000000,0.0,6.0
5,5.718216e-06,-3.337570e-07,7.0,False,0.438043,0.177081,0.069323,0.215457,0.136660,0.058753,...,3.0,4.0,1.0,0.003488,0.032111,8.217204e-03,0000007.bin,0.393146,0.0,2100007.0
6,0.000000e+00,-4.990015e-10,8.0,False,0.000000,0.024843,0.492813,0.657701,0.110455,0.000000,...,3.0,7.0,1.0,0.037960,0.000000,6.195017e-04,0000008.bin,-1.000000,0.0,8.0
7,7.087217e-09,-2.932458e-08,10.0,False,0.018350,0.033251,0.130162,0.381586,0.222467,0.013577,...,2.0,6.0,1.0,0.023995,0.000261,1.032598e-03,0000010.bin,0.205764,0.0,700010.0
8,1.371793e-08,3.786168e-09,11.0,False,0.079462,0.084492,0.093278,0.324455,0.211454,0.007644,...,3.0,4.0,1.0,0.556784,0.000062,2.711810e-04,0000011.bin,0.083947,0.0,2100011.0
9,9.646483e-10,4.714798e-09,12.0,True,0.028829,0.015754,0.136068,0.340661,0.180089,0.007188,...,2.0,6.0,1.0,0.001430,0.000051,9.376263e-05,0000012.bin,0.553938,0.0,12.0


In [86]:
test.shape